In [1]:
from fins3666.defines import *
from fins3666.utils import load_dataframe, format_market_data, current_fx_data
from fins3666.portfolio import Portfolio
from fins3666.models.carry_trade import generate_orders


fx = load_dataframe(os.path.join(DIR_DATA_RAW, 'fx-historical-daily.csv'))
mkt = load_dataframe(os.path.join(DIR_DATA_RAW, 'rates-historical-daily.csv'))

startTime = datetime(2001, 1, 1)

dates = mkt.loc[mkt['Timestamp'] >= startTime, 'Timestamp']
endTime = dates.iloc[-1]


fx_start = pd.DataFrame(current_fx_data(fx, startTime))


startingAssets = [
    {'asset': 'USD', 'units': ACCOUNT_SIZE_USD, 'unit_value_USD': 1.0}]

date = startTime
currentData = current_fx_data(fx, date)

SamplePortfolio = Portfolio(startTime, startingAssets, currentData)


print(SamplePortfolio)
prev_month = startTime.month
monthcounter = 0
for date in dates:
    month = date.month
    yields = mkt[mkt['Timestamp'] <= date]

    fx_rates = current_fx_data(fx, date)

    SamplePortfolio.updateMarketData(
        currentData=fx_rates, yields=yields)

    # Loop that runs once per month
    if (month != prev_month):
        prev_month = month
        # determine which assets to long buy and which to short
        current_positions = SamplePortfolio.summary()
        fx_arrays = SamplePortfolio.forex_spreads

        newOrders = generate_orders(
            yields=yields, current_positions=SamplePortfolio.summary(), fx=fx_arrays)

        for order in newOrders:
            SamplePortfolio.new_order(order)

        monthcounter += 1
        print(SamplePortfolio)

    SamplePortfolio.update(date)


print(SamplePortfolio)



                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00    12,000,000     $     12,000,000.00             Long         100.00%      100.00%

Portfolio - 1 assets
Timestamp: 2001-01-01 00:00:00,
Net Value ($USD): $12,000,000.00


                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00    12,000,000     $     12,000,000.00             Long         100.00%      100.00%

Portfolio - 1 assets
Timestamp: 2001-01-31 00:00:00,
Net Value ($USD): $12,000,000.00


/mnt/c/Users/green/OneDrive - UNSW/Subjects/Current Subjects/FINS3666/Assignment03/src/fins3666/portfolio/portfolio.py:409: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0541099' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self.ledger.at[asset, 'YieldPA'] = yield_pa[0]




                                         Portfolio Overview                                         
Asset    Value      Units          Total Value (USD)           Position       Yield p.a      Weight
USD      $1.00    11,981,741     $     11,981,741.21             Long           5.41%      100.51%
NOK      $0.11    35,074,206     $      3,961,709.83             Long           4.47%       33.23%
NZD      $0.43     8,961,146     $      3,881,967.57             Long           5.23%       32.56%
CAD      $0.65     5,990,786     $      3,888,353.35             Long           3.89%       32.62%
CHF      $0.60     6,536,039    -$      3,950,700.40            Short           3.49%      -33.14%
JPY      $0.01   461,679,175    -$      3,938,738.01            Short           1.40%      -33.04%
SEK      $0.10    38,016,217    -$      3,903,102.41            Short           4.86%      -32.74%

Portfolio - 7 assets
Timestamp: 2001-02-28 00:00:00,
Net Value ($USD): $11,921,231.14


               

ValueError: Units must be greater than zero. Specify order='sell' for sale

In [2]:
'''
                               Portfolio Overview                               
====================================================================================================
Asset    Value      Units      Total Value (USD)             Position      Yield p.a        Weight    
====================================================================================================
USD      $1.00    13019936     $ 13,019,936.00                 Long         560.30%         99.96%
AUD      $0.57       10775     $     6,151.93                  Long         511.16%         0.05%
JPY      $0.01      345638    -$     2,801.42                 Short         150.29%        -0.02%
NZD      $0.49         162     $        79.65                  Long         533.01%         0.00%
CHF      $0.66        2035     $     1,336.74                  Long         121.98%         0.01%



'''

'\n                               Portfolio Overview                               \n====================================================================================================\nAsset    Value      Units      Total Value (USD)             Position      Yield p.a        Weight    \n====================================================================================================\nUSD      $1.00    13019936     $ 13,019,936.00                 Long         560.30%         99.96%\nAUD      $0.57       10775     $     6,151.93                  Long         511.16%         0.05%\nJPY      $0.01      345638    -$     2,801.42                 Short         150.29%        -0.02%\nNZD      $0.49         162     $        79.65                  Long         533.01%         0.00%\nCHF      $0.66        2035     $     1,336.74                  Long         121.98%         0.01%\n\n\n\n'